# Raspagem de Dados com Selenium

Raspar os dados do [ClimaTempo](https://www.climatempo.com.br/previsao-do-tempo/fim-de-semana/cidade/558/saopaulo-sp)

## Configuração do notebook

In [1]:
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from lib.database import Database
from selenium import webdriver
from datetime import datetime
import pandas as pd
import json

DB_NAME = 'db.sqlite3'

# Funções auxiliares
def date_from_timestamp(timestamp):
    return timestamp.strftime('%Y-%m-%d')

def today():
    return datetime.today().strftime('%Y-%m-%d')

## Database Setup

In [2]:
db = Database(DB_NAME)
with db as connection:
    with open('data/locais.csv') as file:
        db.load_locais(file)

## Selenium Setup

In [3]:
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

OBS: o código abaixo está muito extenso e deve ser refatorado

In [4]:
with db as connection:
    # pega lista de locais
    locais = db.get_locais()
    dt_acesso = today()
    
    for local in locais:
        # acessa a URL do local
        driver.get(local['url'])
        
        # clica no botão 15 dias
        driver.find_element(By.ID, 'Botao_barra_navegacao_15_dias').click()
        
        
        print(f"Acesso no dia {dt_acesso} da cidade {local['nome']}:")
        for i in range(1,4):
            id = 'wrapper-chart-' + str(i)
            element = driver.find_element(By.ID, id)
            
            infos = json.loads(element.get_attribute('data-infos'))

            # registra o acesso no banco de dados
            dt_previsao = infos[0]['date'].split()[0]
            try:
                id_acesso = db.registra_acesso(local['id'], dt_acesso, dt_previsao)
                print(f"\tPrevisão para o dia {dt_previsao} feita!")
            except:
                print(f"\tJá existe previsão para o dia {dt_previsao}!")
                continue
                
            for obj in infos:
                db.registra_previsao(id_acesso, obj['date'].split()[1], obj['temperature']['temperature'])
                
driver.quit()

Acesso no dia 2022-08-30 da cidade São Paulo:
	Já existe previsão para o dia 2022-08-30!
	Já existe previsão para o dia 2022-08-31!
	Já existe previsão para o dia 2022-09-01!
Acesso no dia 2022-08-30 da cidade Salvador:
	Já existe previsão para o dia 2022-08-30!
	Já existe previsão para o dia 2022-08-31!
	Já existe previsão para o dia 2022-09-01!


# Exportando o banco para CSV

In [5]:
def export_db_to_csv(path_to_query_file):
    with open(path_to_query_file) as file:
        query = file.read()
    
    with db as connection:
        result = db.con.cursor().execute(query).fetchall()

In [13]:
# Exporta informações das previsões de todos os horarios pra todos os locais
query=''
with open('queries/all_previsoes.sql') as file:
    query = file.read()
    
with db as connection:
    db_df = pd.read_sql_query(query, db.con)
    db_df.to_csv('database.csv', index=False)
    
for group, frame in db_df.groupby(by=['DT_PREVISAO', 'CIDADE', 'HORARIO']):
    tmps = frame['TEMPERATURA'].unique()
    qt_tmps = len(tmps)
    if qt_tmps > 1:
        print(f"{group} teve {tmps}")

('2022-08-30', 'São Paulo', '00:00:00') teve [10 11]
('2022-08-30', 'São Paulo', '01:00:00') teve [11 12]
('2022-08-30', 'São Paulo', '02:00:00') teve [12 13]
('2022-08-30', 'São Paulo', '05:00:00') teve [12 13]
('2022-08-30', 'São Paulo', '06:00:00') teve [12 13]
('2022-08-30', 'São Paulo', '08:00:00') teve [12 13]
('2022-08-30', 'São Paulo', '09:00:00') teve [12 13]
('2022-08-30', 'São Paulo', '10:00:00') teve [13 14]
('2022-08-30', 'São Paulo', '17:00:00') teve [14 15]
('2022-08-30', 'São Paulo', '21:00:00') teve [10 11 13]
('2022-08-30', 'São Paulo', '22:00:00') teve [10 11 12]
('2022-08-30', 'São Paulo', '23:00:00') teve [10 11 12]
('2022-08-31', 'Salvador', '00:00:00') teve [21 22]
('2022-08-31', 'Salvador', '02:00:00') teve [20 21]
('2022-08-31', 'Salvador', '08:00:00') teve [22 23]
('2022-08-31', 'Salvador', '09:00:00') teve [22 23]
('2022-08-31', 'Salvador', '10:00:00') teve [22 23]
('2022-08-31', 'Salvador', '11:00:00') teve [22 24]
('2022-08-31', 'Salvador', '12:00:00') teve